In [1]:
# imports we are going to use for preprocessing

#imports 
import pandas as pd
import numpy as np
import math
from praw.models import MoreComments
from nltk.corpus import stopwords
#from nltk.tokenize import RegexTokenizer
#from util.tokenize import SentTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
#from nltk import FreqDistq
import spacy

import string
import demoji
demoji.download_codes()
from nltk.tokenize import TweetTokenizer

/var/folders/xk/xzsh5hpj1tj6xwdckxm3yzfc0000gn/T/ipykernel_39506/4074095023.py:18: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [15]:
# pyforest imports all the usual libraries we use Pandas , sckitlearn, numpy etc
#import pyforest
import emoji
import re
import en_core_web_sm
from pandas import DataFrame

pd.set_option('display.max_colwidth', None)

In [56]:
df_siemens = pd.read_csv('siemens_twint.txt', delimiter = "\t",  index_col=False)

df_siemens


,"1422515591290793986 2021-08-03 04:12:00 -0700 <Siemens> When it comes to developing &amp; #manufacturing #vaccines, there’s no time to waste! How #DigitalTransformation is helping #pharmaceutical company GSK speed up its processes: 👉 https://t.co/tV6DN82C3t #DigitalTwin"
0,1422146251047440387 2021-08-02 03:44:23 -0700 <Siemens> @SickLizardVFX 🙈 Danke fürs Aufpassen und Melden. Viele Grüße vom Siemens ComsTeam
1,1422131667985211394 2021-08-02 02:46:26 -0700 <Siemens> @YasharAzad all the best :)
2,"1422128369873915904 2021-08-02 02:33:19 -0700 <Siemens> @baidsangeeta Thank you for reaching out! Sounds like you have a request regarding household appliances? That is not us, but a different company with a very similar name: Siemens Home. Please find your local contact at their website: https://t.co/nOIyzmGXTp"
3,1422127534112661505 2021-08-02 02:30:00 -0700 <Siemens> Is your business preparing for the future? Click to explore trends shaping the coming decade and benchmark your progress against other businesses. Find your path to 2030 with us! ✳️ #TransformTheEveryday
4,"1422104886024544261 2021-08-02 01:00:00 -0700 <Siemens> Driven by strong #sustainability goals, the #datacenter industry is at the forefront of exploring cleaner technologies to turn operations green 🌱 Here’s our approach to decarbonizing the data center industry ✏ #decarbonization"
...,...
7094,"248418125303930880 2012-09-19 06:47:51 -0700 <Siemens> Peter Löscher: ""#TheCrystal is a center for communicating with the public. We have 50.000 fans on its #Facebook page."" @thecrystalorg"
7095,247981150923022336 2012-09-18 01:51:28 -0700 <Siemens> Tomorrow marks the opening of the Crystal ( http://t.co/E3ioy7UU). Stay tuned for more news! #Siemens @thecrystalorg http://t.co/yY35oNAK
7096,247655660869779456 2012-09-17 04:18:05 -0700 <Siemens> Welcome to the official corporate twitter channel of #Siemens - your source for news on the company in the Twittersphere!
7097,[!] No more data! Scraping will stop now.


In [12]:
#function to clear spelling mistakes


def prune_multple_consecutive_same_char(transient_tweet_text):
	'''
	yesssssssss  is converted to yess 
	ssssssssssh is converted to ssh
	'''
	transient_tweet_text = re.sub(r'(.)\1+', r'\1\1', transient_tweet_text)
	return transient_tweet_text

In [65]:
result = prune_multple_consecutive_same_char(str(df_siemens))

result

'  1422515591290793986 2021-08-03 04:12:00 -0700 <Siemens> When it comes to developing &amp; #manufacturing #vaccines, there’s no time to waste! How #DigitalTransformation is helping #pharmaceutical company GSK speed up its processes: 👉  https://t.co/tV6DN82C3t #DigitalTwin\n0  1422146251047440387 2021-08-02 03:44:23 -0700 <Siemens> @SickLizardVFX 🙈 Danke fürs Aufpassen und Melden. Viele Grüße vom Siemens ComsTeam\n1  1422131667985211394 2021-08-02 02:46:26 -0700 <Siemens> @YasharAzad all the best :)\n2  1422128369873915904 2021-08-02 02:33:19 -0700 <Siemens> @baidsangeeta Thank you for reaching out! Sounds like you have a request regarding household appliances? That is not us, but a different company with a very similar name: Siemens Home. Please find your local contact at their website:  https://t.co/nOIyzmGXTp\n3  1422127534112661505 2021-08-02 02:30:00 -0700 <Siemens> Is your business preparing for the future? Click to explore trends shaping the coming decade and benchmark your pro

In [66]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
print(result)

  1422515591290793986 2021-08-03 04:12:00 -0700 <Siemens> When it comes to developing &amp; #manufacturing #vaccines, there’s no time to waste! How #DigitalTransformation is helping #pharmaceutical company GSK speed up its processes: 👉  https://t.co/tV6DN82C3t #DigitalTwin
0  1422146251047440387 2021-08-02 03:44:23 -0700 <Siemens> @SickLizardVFX 🙈 Danke fürs Aufpassen und Melden. Viele Grüße vom Siemens ComsTeam
1  1422131667985211394 2021-08-02 02:46:26 -0700 <Siemens> @YasharAzad all the best :)
2  1422128369873915904 2021-08-02 02:33:19 -0700 <Siemens> @baidsangeeta Thank you for reaching out! Sounds like you have a request regarding household appliances? That is not us, but a different company with a very similar name: Siemens Home. Please find your local contact at their website:  https://t.co/nOIyzmGXTp
3  1422127534112661505 2021-08-02 02:30:00 -0700 <Siemens> Is your business preparing for the future? Click to explore trends shaping the coming decade and benchmark your progress

In [14]:
result = result.split("\n")
result

['                                                 1422515591290793986 2021-08-03 04:12:00 -0700 <Siemens> When it comes to developing &amp; #manufacturing #vaccines, there’s no time to waste! How #DigitalTransformation is helping #pharmaceutical company GSK speed up its processes: 👉   #DigitalTwin',
 '0                                                                                                                                                                                     1422146251047440387 2021-08-02 03:44:23 -0700 <Siemens> @SickLizardVFX 🙈 Danke fürs Aufpassen und Melden. Viele Grüße vom Siemens ComsTeam',
 '1                                                                                                                                                                                                                                            1422131667985211394 2021-08-02 02:46:26 -0700 <Siemens> @YasharAzad all the best :)',
 '2     1422128369873915904 2021-08-02 02:33:19 

In [21]:
def to_LowerCase(transient_tweet_text):
    '''
	Convert tweet text to lower to lower case alphabets
	'''
    transient_tweet_text = transient_tweet_text.lower()
    return transient_tweet_text

In [55]:
new_result = to_LowerCase(result)

new_result

AttributeError: 'DataFrame' object has no attribute 'lower'

In [25]:
#removing all emojis

def remove_emoji(transient_tweet_text):
    '''
    replace emoji with the respective emotion
    '''
    tweet_tokenizer = TweetTokenizer()
    tokenized_tweet = tweet_tokenizer.tokenize(transient_tweet_text)
    emojis_present = demoji.findall(transient_tweet_text)
    tweet_no_emoji=''
    for i,s in enumerate(tokenized_tweet):
        if s in emojis_present.keys():
            tweet_no_emoji = tweet_no_emoji + ' ' + emojis_present[s]
        else:
            tweet_no_emoji = tweet_no_emoji + ' ' + s
    return tweet_no_emoji

In [26]:
new_result = remove_emoji(new_result)

new_result

' 0   1422515591 2907939 86 2021-08- 03 04:12 : 00 - 0700 <siemens> when it comes to developing & #manufacturing #vaccines , there ’ s no time to waste ! how #digitaltransformation is helping #pharmaceutical company gsk speed up its processes : backhand index pointing right #digitaltwin 1 0   1422146251 04744038 7 2021-08- 02 03:44 : 23 - 0700 <siemens> @sicklizardvfx see-no-evil monkey danke fürs aufpassen und melden . viele grüße vom siemens comsteam 2 1   1422131667 9852113 94 2021-08- 02 02:46 : 26 - 0700 <siemens> @yasharazad all the best :) 3 2 14221283698 7391590 4 2021-08- 02 02:33 : 19 - 0700 <siemens> @baidsangeeta thank you for reaching out ! sounds like you have a request regarding household appliances ? that is not us , but a different company with a very similar name : siemens home . please find your local contact at their website : 4 3 14221275341 12661505 2021-08- 02 02:30 : 00 - 0700 <siemens> is your business preparing for the future ? click to explore trends shaping 

In [29]:
#stripping unicode text
def strip_unicode(transient_tweet_text):
    '''
    Strip all unicode characters from a tweet
    '''
    tweet = ''.join(i for i in transient_tweet_text if ord(i)<128)
    return tweet 

In [30]:
clean_tweets = strip_unicode(new_result)

clean_tweets

' 0   1422515591 2907939 86 2021-08- 03 04:12 : 00 - 0700 <siemens> when it comes to developing & #manufacturing #vaccines , there  s no time to waste ! how #digitaltransformation is helping #pharmaceutical company gsk speed up its processes : backhand index pointing right #digitaltwin 1 0   1422146251 04744038 7 2021-08- 02 03:44 : 23 - 0700 <siemens> @sicklizardvfx see-no-evil monkey danke frs aufpassen und melden . viele gre vom siemens comsteam 2 1   1422131667 9852113 94 2021-08- 02 02:46 : 26 - 0700 <siemens> @yasharazad all the best :) 3 2 14221283698 7391590 4 2021-08- 02 02:33 : 19 - 0700 <siemens> @baidsangeeta thank you for reaching out ! sounds like you have a request regarding household appliances ? that is not us , but a different company with a very similar name : siemens home . please find your local contact at their website : 4 3 14221275341 12661505 2021-08- 02 02:30 : 00 - 0700 <siemens> is your business preparing for the future ? click to explore trends shaping the 

In [31]:
#Mentions
def process_Mentions(transient_tweet_text):
    '''
	Identify mentions if any
	'''
    transient_tweet_text = re.sub(r"@(\w+)"," ", transient_tweet_text)
    return transient_tweet_text

In [32]:
process_Mentions(clean_tweets)

' 0   1422515591 2907939 86 2021-08- 03 04:12 : 00 - 0700 <siemens> when it comes to developing & #manufacturing #vaccines , there  s no time to waste ! how #digitaltransformation is helping #pharmaceutical company gsk speed up its processes : backhand index pointing right #digitaltwin 1 0   1422146251 04744038 7 2021-08- 02 03:44 : 23 - 0700 <siemens>   see-no-evil monkey danke frs aufpassen und melden . viele gre vom siemens comsteam 2 1   1422131667 9852113 94 2021-08- 02 02:46 : 26 - 0700 <siemens>   all the best :) 3 2 14221283698 7391590 4 2021-08- 02 02:33 : 19 - 0700 <siemens>   thank you for reaching out ! sounds like you have a request regarding household appliances ? that is not us , but a different company with a very similar name : siemens home . please find your local contact at their website : 4 3 14221275341 12661505 2021-08- 02 02:30 : 00 - 0700 <siemens> is your business preparing for the future ? click to explore trends shaping the coming decade and benchmark your pr

In [33]:
def process_HashTags(transient_tweet_text):
    '''
	Strip all Hashtags from a tweet
	'''
    transient_tweet_text = re.sub(r"#(\w+)\b", '', transient_tweet_text)
    return transient_tweet_text

In [34]:
clean_tweet = process_HashTags(clean_tweets)

clean_tweet

' 0   1422515591 2907939 86 2021-08- 03 04:12 : 00 - 0700 <siemens> when it comes to developing &   , there  s no time to waste ! how  is helping  company gsk speed up its processes : backhand index pointing right  1 0   1422146251 04744038 7 2021-08- 02 03:44 : 23 - 0700 <siemens> @sicklizardvfx see-no-evil monkey danke frs aufpassen und melden . viele gre vom siemens comsteam 2 1   1422131667 9852113 94 2021-08- 02 02:46 : 26 - 0700 <siemens> @yasharazad all the best :) 3 2 14221283698 7391590 4 2021-08- 02 02:33 : 19 - 0700 <siemens> @baidsangeeta thank you for reaching out ! sounds like you have a request regarding household appliances ? that is not us , but a different company with a very similar name : siemens home . please find your local contact at their website : 4 3 14221275341 12661505 2021-08- 02 02:30 : 00 - 0700 <siemens> is your business preparing for the future ? click to explore trends shaping the coming decade and benchmark your progress against other businesses . fin

In [44]:
clean_tweet = re.sub("#\S+", " ", clean_tweet)


clean_tweet


' 0   1422515591 2907939 86 2021-08- 03 04:12 : 00 - 0700 <siemens> when it comes to developing &   , there  s no time to waste ! how  is helping  company gsk speed up its processes : backhand index pointing right  1 0   1422146251 04744038 7 2021-08- 02 03:44 : 23 - 0700 <siemens> @sicklizardvfx see-no-evil monkey danke frs aufpassen und melden . viele gre vom siemens comsteam 2 1   1422131667 9852113 94 2021-08- 02 02:46 : 26 - 0700 <siemens> @yasharazad all the best :) 3 2 14221283698 7391590 4 2021-08- 02 02:33 : 19 - 0700 <siemens> @baidsangeeta thank you for reaching out ! sounds like you have a request regarding household appliances ? that is not us , but a different company with a very similar name : siemens home . please find your local contact at their website : 4 3 14221275341 12661505 2021-08- 02 02:30 : 00 - 0700 <siemens> is your business preparing for the future ? click to explore trends shaping the coming decade and benchmark your progress against other businesses . fin

In [50]:
type(clean_tweet)

str

In [45]:
#Removing numbers and whitespace and convert to lowercase

def clean_text(text):
    # remove numbers
    text_nonum = re.sub(r'\d+', '', text)
    # substitute multiple whitespace with single whitespace
    # remove punctuations and convert characters to lower case
    text_nopunct = "".join([char.lower() for char in text_nonum if char not in string.punctuation]) 
    # Also, removes leading and trailing whitespaces
    text_no_doublespace = re.sub('\s+', ' ', text_nopunct).strip()
    return text_no_doublespace

clean_tweets = clean_text(clean_tweets)


clean_tweets

'siemens when it comes to developing there s no time to waste how is helping company gsk speed up its processes backhand index pointing right siemens seenoevil monkey danke frs aufpassen und melden viele gre vom siemens comsteam siemens all the best siemens thank you for reaching out sounds like you have a request regarding household appliances that is not us but a different company with a very similar name siemens home please find your local contact at their website siemens is your business preparing for the future click to explore trends shaping the coming decade and benchmark your progress against other businesses find your path to with us siemens driven by strong goals the industry is at the forefront of exploring cleaner technologies to turn operations green seedling here s our approach to decarbonizing the data center industry pencil siemens peter lscher is a center for communicating with the public we have fans on its page siemens tomorrow marks the opening of the crystal stay t